In [22]:
from src.data.datasets import SiteDataset
import jax.numpy as jnp
site_data = SiteDataset("5a0546857ecc773753327266")

floor = site_data.floors[0]
trace = floor.traces[0]
matrices = trace.matrices
position = matrices["position"]
wifi = matrices["wifi"]
from numpyro import handlers
import numpyro.distributions as dist
from numpyro import sample, plate

In [2]:
height, width = floor.info["map_info"]["height"], floor.info["map_info"]["width"]

In [3]:
floor_uniform = dist.Independent(
    dist.Uniform(low=jnp.array([0, 0]), high=jnp.array([height, width])), 1
)


In [17]:
with handlers.seed(rng_seed=0):
    print(sample("location", floor_uniform))

[117.9239   80.87863]


In [41]:
(~jnp.isnan(position[:, 0])).sum()

DeviceArray(4, dtype=int32)

In [43]:
def initial_model(position, wifi):

    T = position.shape[0]
    K = wifi.shape[1]

    location_observed = ~jnp.isnan(position[:, 0])

    sigma_eps = 0.1
    sigma = 0.1

    mu_omega_0 = -45
    sigma_omega_0 = 5
    sigma_omega = 2

    with plate("wifis", K):
        omega_0 = sample("omega_0", dist.Normal(mu_omega_0, sigma_omega_0))
        wifi_locations = sample("wifi_location", floor_uniform)

    x = jnp.zeros((T, 2))

    x0 = sample("x_0", floor_uniform)

    for t in range(1, T):
        x[t, :] = sample(f"x_{1}", dist.Normal(x[t - 1, :], sigma))

    with plate("x_obs", sum(location_observed)):
        x_obs = sample(
            f"x_obs",
            dist.Normal(x[location_observed, :], sigma_eps),
            obs=position[location_observed, :],
        )

#     beta = numpyro.sample("beta", dist.Normal(0, 1).expand((2)))

#     sigma = numpyro.sample("sigma", dist.HalfCauchy(1))
#     tau = numpyro.sample("tau", dist.HalfCauchy(1))

#     z = np.zeros((T + T_forecast))
#     y = np.zeros((T + T_forecast))

#     z[0, :] = numpyro.sample("z0", dist.Normal(0.05, 0.1))
#     z[1, :] = numpyro.sample("z1", dist.Normal(beta[:, 0]*z[0, :]))

#     for i in range(2, T):
#         z[i, :] = numpyro.sample(f"z{i}", dist.Normal(beta[:, 0]*z[i-1, :] + beta[:, 0]*z[i-2, :], tau))

#     y[:T, :] = numpyro.sample("y", dist.Normal(z[:T, :], sigma), obs=obs)

#     for i in range(T, T + T_forecast):
#         z[i, :] = numpyro.sample(f"z{i}", dist.Normal(beta[:, 0]*z[i-1, :] + beta[:, 0]*z[i-2, :], tau ))

#     y[T:, :] = numpyro.sample("y_forecast", dist.Normal(z[T:, :], sigma))

with handlers.seed(rng_seed=0):
    initial_model(position, wifi)


TypeError: '<class 'jax.interpreters.xla._DeviceArray'>' object does not support item assignment. JAX arrays are immutable; perhaps you want jax.ops.index_update or jax.ops.index_add instead?